## Phase 1 Pipeline

In [1]:
from model_training import *

In [8]:
hyperparam_space = {
    'window_size': [180],
    'model': 'lstm',
    'lstm': {
      'units': [10],
      'activation': ['relu'],
      'recurrent_regularizer': ['l2'],
      'kernel_regularizer': ['l2']
    },
    'dense': {
        'activation': ['tanh'],
        'kernel_regularizer': ['l2']
    },
    'optimizer': {
        'loss': ['mean_absolute_error'],
        'optimizer': ['adam']
    }
}

In [9]:
hyperparameter_configs = get_model_configs(hyperparam_space)
print(hyperparameter_configs)

[{'window_size': 180, 'dense': {'activation': 'tanh', 'kernel_regularizer': 'l2'}, 'optimizer': {'loss': 'mean_absolute_error', 'optimizer': 'adam'}, 'lstm': {'units': 10, 'activation': 'relu', 'recurrent_regularizer': 'l2', 'kernel_regularizer': 'l2'}}]
